# Search feature weighting

This notebook computes feature weights for a search ranking model.

As of early 2020, search ranking was being done by an ad-hoc pairwise comparison function that may not even be transitive. We want to replace it with a more structured and analyzable approach that can additional search features besides corpus frequency, such as cosine vector distance for now, with room for more features later.

The basic pieces are:
  - Use occurrence of a search result in the search survey as a 0-or-1 relevance variable
  - Create a relevance score from that using some fairly basic linear modelling techniques to compute best-fit feature weights
  - Measure success by the three10 score: the mean percentage of how many top-3 results from the linguist survey appear in the top 10 search results

There are many interesting possible future improvements here, such as:
  - Use occurence anywhere in sample, instead of top3 results only, for training
  - More precise training data, e.g., relevance rankings of 1-5
  - Handle homonyms in training data instead of matching purely on wordform text
  - More training data, specifically how many results per query we have human scores for
  - More features, e.g., tf-idf
  - Higher quality features, e.g., better stopword filtering in vector computations
  - Map features to have similar ranges and distributions to better allow the regression to more effectively compare them
  - Separate training and test sets
  - Fancier models
  - Better evaluation functions, such as discounted cumulative gain

That said, having all the pieces together, even in a very basic form, is already an improvement over the existing search, so let’s start with that.

## Preliminaries

Load some libraries. `weighting_nb_code.py` contains some more python-y code that was extracted from some exploratory jupyter notebooks once it was working ok.

In [1]:
import importlib


import numpy as np
import pandas as pd
import statsmodels.formula.api as smf

import weighting_nb_code

# Reload the code in `weighting_nb_code.py` by re-running this cell, or
# by copying the next line into other cells. If this reload mechanism
# proves insufficient, there is also `IPython.lib.deepreload`.
importlib.reload(weighting_nb_code);

/Users/jolenepoulin/.local/share/virtualenvs/morphodict-5QYDyuRz/lib/python3.9/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


First, if the JSON output file doesn’t already exist, we’ll run the `featuredump` management command to get our raw data. CVD search is not yet on by default, so we add a fancy query to enable it.

In [2]:
![ -f sample-features.json ] || \
    {weighting_nb_code.BASE_DIR.parent.parent}/crkeng-manage featuredump \
        --prefix-queries-with 'cvd:retrieval' \
        > sample-features.json

100%|█████████████████████████████████████████| 548/548 [00:39<00:00, 13.76it/s]


The loaded feature data looks like this:

In [3]:
data = weighting_nb_code.dataframe_from_featuredump('sample-features.json')
data

,cosine_vector_distance,relevance_score,query,source_language_keyword_match,morpheme_ranking,webapp_sort_rank,pos_match,query_wordform_edit_distance,analyzable_inflection_match,lemma_wordform_text,wordform_text,wordform_length,source_language_match,corp_freq,is_lemma,source_language_affix_match,target_language_keyword_match,target_language_affix_match,definitions
0,0.000000,0.202368,about,[],None,1,0.0,NaN,None,ohci,ohci,4,None,3,True,None,[about],True,"[[from there, thence, out of, CW], [with, by m..."
1,0.287011,0.178594,about,[],None,2,0.0,NaN,None,kêkâc,kêkâc,5,None,3,True,None,[about],True,"[[almost, just about, nearly, CW], [Almost, MD..."
2,0.000000,0.155420,about,[],None,3,0.0,NaN,None,êtikwê,êtikwê,6,None,2,True,None,[about],True,"[[apparently, I guess, I suppose, supposedly, ..."
3,0.302491,0.130520,about,[],None,4,0.0,NaN,None,papâmipayiw,papâmipayiw,11,None,2,True,None,[about],True,"[[s/he rides around, s/he rides about (by car ..."
4,0.305036,0.130336,about,[],None,5,0.0,NaN,None,kiskêyimêw,kiskêyimêw,10,None,2,True,None,[about],True,"[[s/he knows s.o., s/he knows about s.o., CW]]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71964,NaN,-0.008380,they see us,[],None,145,0.0,NaN,None,ocâpahtowin,ocâpahtowin,11,None,0,True,None,[see],None,"[[being alive to see one another, CW]]"
71965,NaN,-0.008380,they see us,[],None,146,0.0,NaN,None,otisâpahcikêw,otisâpahcikêw,13,None,0,True,None,[see],None,"[[s/he has lived long enough to see things, CW]]"
71966,NaN,-0.008380,they see us,[],None,147,0.0,NaN,None,otisâpamiwêw,otisâpamiwêw,12,None,0,True,None,[see],None,"[[s/he has lived long enough to see people, CW]]"
71967,NaN,-0.008380,they see us,[],None,148,0.0,NaN,None,wâpahtamowin,wâpahtamowin,12,None,0,True,None,[see],None,"[[vision, seeing, sight, CW]]"


Here’s the current combined result survey sample.

In [4]:
weighting_nb_code.survey()

,Query,Nêhiyawêwin 1,Nêhiyawêwin 2,Nêhiyawêwin 3
0,about,wayês,ohci,papâ
1,all,kahkiyaw,kapê,mâwaci
2,also,mîna,êkwa,kisik
3,and,êkwa,mîna,kisik
4,as,kisik,wiya,tâpiskôt
...,...,...,...,...
543,she sees him,wâpamêw,NaN,NaN
544,starblanket,atâhkakohp,acâhkosa kâ-otakohpit,NaN
545,star blanket,atâhkakohp,acâhkosa kâ-otakohpit,NaN
546,being taught,kiskinwahamâkosiw,NaN,NaN


And `weighting_nb_code.py` contains a function to annotate the `featuredump` results with the top3/three10 metrics.

In [5]:
weighting_nb_code.top3_and_310_stats(data, rank_column="webapp_sort_rank")[
    ["query", "wordform_text", "definitions", "actual_result_ranks", "top3", "three10"]
]

/Users/jolenepoulin/.local/share/virtualenvs/morphodict-5QYDyuRz/lib/python3.9/site-packages/pandas/core/algorithms.py:794: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


,query,wordform_text,definitions,actual_result_ranks,top3,three10
0,"""horse""",misatim,"[[horse, CW]]","[2.0, 3.0]",100.0,100.0
1,'horse',misatim,"[[horse, CW]]","[2.0, 5.0]",100.0,100.0
2,Calgary,otôskwanihk,"[[Calgary, AB, CW]]",[4.0],100.0,100.0
3,Cree,nêhiyaw,"[[Nehiyaw, CW], [Cree, Cree man, First Nations...","[2.0, 4.0]",100.0,100.0
4,Cree language,nêhiyawêwin,"[[the Cree language, CW], [speaking Cree, CW],...",[1.0],100.0,100.0
...,...,...,...,...,...,...
543,yellow hat,osâwastotin,"[[yellow hat, CW]]",[3.0],100.0,100.0
544,you,kiya,"[[you, CW]]","[1.0, 2.0, 8.0]",100.0,100.0
545,young,oski,NaN,[],0.0,0.0
546,younger sibling,nisîmis,"[[my younger sibling (brother or sister), my y...",[3.0],100.0,100.0


## Initial results from dictionary code

Without any cosine-vector stuff, here are the current search stats we want to beat. 81.3% for top3, and 59.4% for three10.

In [6]:
import os
if os.path.isfile('sample-features-orig.json'):
    data_orig = weighting_nb_code.dataframe_from_featuredump('sample-features-orig.json')
    display(weighting_nb_code.top3_and_310_stats_summary(data_orig, rank_column="webapp_sort_rank"))

Note: this won’t exactly match what the django `/search-quality` pages report, because of some differences in determining exactly what the rank is. In the django code, if the results are `(non-lemma1, non-lemma2)`, we count the ranks as `(1, 3)` because the UI display of `non-lemma1` includes its lemma definition at rank 2. Here we skip that for now, but the results should be close enough.

And, for comparison, here are the stats when we added a very basic cosine vector distance model to the search:

In [7]:
weighting_nb_code.top3_and_310_stats_summary(data, rank_column="webapp_sort_rank")

/Users/jolenepoulin/.local/share/virtualenvs/morphodict-5QYDyuRz/lib/python3.9/site-packages/pandas/core/algorithms.py:794: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


top3       81.630170
three10    72.110706
dtype: float64

The top3 score—what percent of desired search results we see anywhere in the list—has gone up. That is, the vector model’s ability to resolve synonyms has improved recall. But the three10 score—what percent of desired search results are near the top—has gone down since we don’t have a good ranking mechanism.

## Modelling

At this point we have all the definition and feature data from the webapp loaded, and we could experiment by adding more data columns with additional features. Those features could be computed by Python code here, or loaded from data files.

For this first version, let’s stick with what we have:

In [13]:
def prep_results_for_regression(df):
    # The default value used for `fillna()` doesn’t matter if we
    # also have an indicator variable, but things get trickier
    # with logarithms.
    return df.assign(
        morpheme_ranking=df["morpheme_ranking"].fillna(1),
        has_morpheme_ranking=weighting_nb_code.has_col_as_int(df, "morpheme_ranking"),
        has_cosine_vector_distance=weighting_nb_code.has_col_as_int(df, "cosine_vector_distance"),
        cosine_vector_distance=df["cosine_vector_distance"].fillna(1.1),
        is_in_survey=df.apply(weighting_nb_code.is_in_survey, axis=1),
        keyword_match_len=df['target_language_keyword_match'].apply(len),
        pos_match=df["pos_match"].fillna(0),
        doc_freq=df["corp_freq"].fillna(0),
    )

In [16]:
df = prep_results_for_regression(data)
results = smf.ols(
    """
    is_in_survey ~
        wordform_length
        + has_morpheme_ranking
        + morpheme_ranking
        + pos_match
        + doc_freq
        + np.log(1 + cosine_vector_distance)
    """,
    data=df,
).fit()
display(results.summary())
sorted_results = weighting_nb_code.rank_by_predictor(df, results)
weighting_nb_code.top3_and_310_stats_summary(sorted_results, rank_column="result_rank")

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           is_in_survey   R-squared:                       0.084
Model:                            OLS   Adj. R-squared:                  0.084
Method:                 Least Squares   F-statistic:                     1649.
Date:                Fri, 25 Mar 2022   Prob (F-statistic):               0.00
Time:                        13:34:09   Log-Likelihood:                 64845.
No. Observations:               71969   AIC:                        -1.297e+05
Df Residuals:                   71964   BIC:                        -1.296e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
======================================================================================================
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Intercept                              0.0244      0.001     30.789      0.000       0.023       0.026
wordform_length                       -0.0004      0.000     -3.984      0.000      -0.001      -0.000
has_morpheme_ranking               -2.094e-17    4.6e-19    -45.490      0.000   -2.18e-17      -2e-17
morpheme_ranking                       0.0244      0.001     30.789      0.000       0.023       0.026
pos_match                              0.0019      0.002      0.822      0.411      -0.003       0.006
doc_freq                               0.0447      0.001     59.087      0.000       0.043       0.046
np.log(1 + cosine_vector_distance)    -0.0693      0.002    -39.887      0.000      -0.073      -0.066
==============================================================================
Omnibus:                    99437.113   Durbin-Watson:                   1.860
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         18001220.642
Skew:                           8.360   Prob(JB):                         0.00
Kurtosis:                      78.653   Cond. No.                     2.41e+18
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.67e-30. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

/Users/jolenepoulin/.local/share/virtualenvs/morphodict-5QYDyuRz/lib/python3.9/site-packages/pandas/core/algorithms.py:794: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


top3       81.630170
three10    67.639903
dtype: float64

Objectively, we have increased how many of the top survey results we display at all from 81.3% to 83.4%, and we have increased the mean number of them that appear in the top 10 results per query from ~60% to ~70%. That’s great!

Let’s take a look at a sample query. Before:

In [17]:
(weighting_nb_code.top3_and_310_stats(data, rank_column='webapp_sort_rank')
     .query('query == "counts"'))[['query', 'actual_result_ranks', 'top3', 'three10']]

/Users/jolenepoulin/.local/share/virtualenvs/morphodict-5QYDyuRz/lib/python3.9/site-packages/pandas/core/algorithms.py:794: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


,query,actual_result_ranks,top3,three10
126,counts,"[1.0, 2.0, 5.0]",100.0,100.0


When searching for ‘counts,’ before all the good results were showing up somewhere in the results, but none of them were in the top 10.

Now, with this new ranking model, the top results from the survey show up at the top of the search results, and even in 1, 2, 3 order:

In [18]:
(weighting_nb_code.top3_and_310_stats(sorted_results, rank_column='result_rank')
     .query('query == "counts"'))[['query', 'actual_result_ranks', 'top3', 'three10']]

/Users/jolenepoulin/.local/share/virtualenvs/morphodict-5QYDyuRz/lib/python3.9/site-packages/pandas/core/algorithms.py:794: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


,query,actual_result_ranks,top3,three10
126,counts,"[1.0, 2.0, 5.0]",100.0,100.0


If we look in more detail at the results, we can see that the cosine vector distance and the morpheme ranking are being combined, but one doesn’t overrule the other. Rarer words generally appear later in the list, but a strong CVD score can move it earlier, and vice versa.

In [19]:
sorted_results.query("query == 'counts'").sort_values('score', ascending=False)[
    ['wordform_text', 'definitions', 'morpheme_ranking',
     'has_cosine_vector_distance',
     'cosine_vector_distance', 'is_in_survey', 'score']
 ].head(10)

,wordform_text,definitions,morpheme_ranking,has_cosine_vector_distance,cosine_vector_distance,is_in_survey,score
53160,akimêw,"[[s/he counts s.o., CW]]",1,1,5.960464e-08,1,0.135747
53159,akihcikêw,"[[s/he counts, CW]]",1,1,5.960464e-08,1,0.134446
53162,itakisow,"[[s/he is counted thus, CW], [it is held in su...",1,0,1.100000e+00,0,0.128185
53161,itakihtêw,"[[it is counted thus, it is valued thus, it co...",1,0,1.100000e+00,0,0.127751
53163,akihtam,"[[s/he counts s.t., CW], [s/he adds s.t., s/he...",1,1,5.960464e-08,1,0.090573
53182,kanawêyimêw,"[[s/he looks after s.o., s/he takes care of s....",1,1,6.323055e-01,0,0.054868
53166,akihtâw,"[[s/he counts s.t., CW]]",1,1,5.960464e-08,0,0.045832
53165,akihtâsow,"[[s/he counts, CW]]",1,1,5.960464e-08,0,0.044965
53169,wiyakimêw,"[[s/he counts s.o., CW], [s/he puts a price on...",1,1,5.960464e-08,0,0.044965
53168,wiyakihtam,"[[s/he counts s.t., CW], [s/he puts a price on...",1,1,5.960464e-08,0,0.044531


This is quite a bit better than only using the morpheme ranking:

In [20]:
if os.path.isfile('sample-features-orig.json'):
    display((data_orig.assign(is_in_survey=data_orig.apply(weighting_nb_code.is_in_survey, axis=1))
     .query("query == 'counts'").sort_values('webapp_sort_rank')[
        ['wordform_text', 'definitions', 'morpheme_ranking', 'is_in_survey']
     ]).head(10))

## Model export

While the model generated by the `statsmodels` library is `pickle`able, since it’s a fairly basic linear model, for now we will just print the parameters to use in the webapp.

In [23]:
print(results.params.to_json(indent=2))

{
  "Intercept":0.0244343126,
  "wordform_length":-0.0004337761,
  "has_morpheme_ranking":-2.094435278e-17,
  "morpheme_ranking":0.0244343126,
  "pos_match":0.0019122512,
  "doc_freq":0.0447407131,
  "np.log(1 + cosine_vector_distance)":-0.0693265378
}


And here are some test vectors for ensuring the implementation is working correctly.

In [28]:
import re

def print_test_vector(**kwargs):
    df = prep_results_for_regression(pd.DataFrame([{
        "query": "counts",
        "wordform_text": "",
        "target_language_keyword_match": [],
        "wordform_length": 0,
        "keyword_match_len": 0,
        "morpheme_ranking": np.nan,
        "cosine_vector_distance": np.nan,
        "pos_match": 0,
        "corp_freq": 0,
        **kwargs
    }]))
    ret = results.predict(df)[0]
    # future python feature “underscore as a decimal separator”
    # https://bugs.python.org/issue43624 would be handy here
    ret = f'{ret:_f}'
    if '.' in ret:
        l, r = ret.split('.')
        r = re.sub(r'(...)(?=.)', r'\1_', r)
        ret = f'{l}.{r}'
    print(ret)

In [29]:
print_test_vector()

-0.002_567


In [30]:
print_test_vector(cosine_vector_distance=0.7)

0.012_082


In [31]:
print_test_vector(morpheme_ranking=12.8)

0.285_758


In [32]:
print_test_vector(cosine_vector_distance=0.7, morpheme_ranking=12.8)

0.300_407


In [37]:
print_test_vector(cosine_vector_distance=0.7, morpheme_ranking=12.8, wordform_length=9, target_language_keyword_match_len=1)

0.296_503


In [36]:
print_test_vector(cosine_vector_distance=0.7, morpheme_ranking=12.8, pos_match=5)

0.309_968
